# Get computing time data to obtain the first solution
This notebook presents computing time data to get the first solution with Seed2Lp, with a timeout at 45 min, in full Network or Target mode, without accumulation allowed, only for subset minimal optimization.


To run correctly this notebook and have the same results as the paper, you must first download the raw results: [https://doi.org/10.57745/OS1JND](https://doi.org/10.57745/OS1JND)

This notebook is written with the hierarchy of downloaded files, if you want to try it with the test form the run notebooks, it is needed to first restructure your data to match the hierarchy of downloaded files.

We suppose here that the downloaded files are in a directory named "analyses", this directory path can be changed to your directory path where the data are saved.

## Requirements
- Download and Install *R*: https://cran.r-project.org/
- Type `R` in your console, the R lanuage will run.
- Execute the command: `install.packages('IRkernel', repos = 'http://cran.us.r-project.org');IRkernel::installspec()`
- Execute the command: `install.packages("reshape2")`
- Execute the command: `install.packages("data.table")`
- Execute the command: `install.packages("ggplot2")`


Install the package rpv2:

In [6]:
!pip install rpy2

  Using cached rpy2-3.5.16.tar.gz (220 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rpy2: filename=rpy2-3.5.16-cp311-cp311-linux_x86_64.whl size=261792 sha256=88a2022583d7fb3ab79ea21354bf08cc3e5e137e1ebc0e57556ab42091e4874e
  Stored in directory: /home/cghassem/.cache/pip/wheels/da/60/76/3bc67cbf19cb7dd4806c73262e7588dfada92f80fcf3558fc5
Successfully built rpy2


# Variables to change (if wanted)

In [6]:
analyse_dir = "../../analyses"
data_dir = f"{analyse_dir}/data"

# Directories to create for R analyses and plots
timer_tables_dir = f"{analyse_dir}/results/timer_one_sol/tables"
timers_plot_dir = f"{analyse_dir}/results/timer_one_sol/plots"

# Initialisation and functions

In [2]:
import os
import pandas as pd

In [7]:
if not os.path.isdir(timer_tables_dir):
    os.makedirs(timer_tables_dir)

if not os.path.isdir(timers_plot_dir):
    os.makedirs(timers_plot_dir)

In [8]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [9]:
result_dir = os.path.join(analyse_dir, "results")
one_sol_results_dir = os.path.join(result_dir, "one_solution")
one_sol_supp_data = os.path.join(result_dir,"supp_data","one_solution_supp_data.tsv")
sbml_dir = f'{data_dir}/bigg/sbml'

In [10]:
def get_timers(file_path, mode):
    table_all = pd.read_csv(file_path, sep='\t', lineterminator='\n')
    table_all=table_all.fillna(-50)
    table_all.loc[table_all["minimize"] == "unsat", "minimize"] = -100
    table_all.loc[table_all["minimize_opti"] == "unsat", "minimize_opti"] = -100
    table_all.loc[table_all["submin"] == "unsat", "submin"] = -100
    table_all.loc[table_all["minimize"] == "Time out", "minimize"] = -200
    table_all.loc[table_all["minimize"] == "time out", "minimize"] = -200
    table_all.loc[table_all["minimize_opti"] == "Time out", "minimize_opti"] = -200
    table_all.loc[table_all["minimize_opti"] == "time out", "minimize_opti"] = -200
    table_all.loc[table_all["submin"] == "Time out", "submin"] = -200
    table_all.loc[table_all["submin"] == "time out", "submin"] = -200
    table_all['minimize'] = table_all['minimize'].astype('float')
    table_all['minimize_opti'] = table_all['minimize_opti'].astype('float')
    table_all['submin'] = table_all['submin'].astype('float')

    timers_all= table_all.loc[table_all["type_data"] == "Solving (sec)"]
    # No accumulation mode
    timers_all = timers_all[timers_all['accumulation']==False]

    if mode == "full":
        return timers_all[timers_all["search_mode"]=="Full network"]
    elif mode == "target":
        return timers_all[timers_all["search_mode"]=="Target"]
    else:
        return timers_all


In [11]:
def get_fluxes(directory:str, mode:str, optim:str=None):
    flux_all=pd.DataFrame(columns=['species', 'biomass_reaction', 'solver_type', 'search_mode',
                                     'search_type', 'accumulation', 'model', 'size', 'lp_flux', 'cobra_flux_init',
                                     'cobra_flux_no_import', 'cobra_flux_seeds', 'cobra_flux_demands',
                                     'has_flux', 'has_flux_seeds', 'has_flux_demands', 'timer'])
    flux_all['accumulation'] = flux_all['accumulation'].astype('bool')
    flux_all['has_flux'] = flux_all['has_flux'].astype('bool')
    flux_all['has_flux_seeds'] = flux_all['has_flux_seeds'].astype('bool')
    flux_all['has_flux_demands'] = flux_all['has_flux_demands'].astype('bool')

    for dirpath, _, filenames in os.walk(directory):
        for filename in [f for f in filenames if (f.endswith("_fluxes.tsv") or f.endswith("_fluxes_from_result.tsv"))]:
            # By default in this notebook we want the no accumulation mode for seed2lp results
            if  "_no_accu_" in filename \
                and   ((mode == "full" and "_fn_" in filename) \
                    or (mode == "target" and "_tgt_" in filename))\
                or mode == "netseed":
                file_path=os.path.join(dirpath, filename)
                current_df = pd.read_csv(file_path, sep='\t', lineterminator='\n')
                current_df['accumulation'] = current_df['accumulation'].astype('bool')
                current_df['has_flux'] = current_df['has_flux'].astype('bool')
                current_df['has_flux_seeds'] = current_df['has_flux_seeds'].astype('bool')
                current_df['has_flux_demands'] = current_df['has_flux_demands'].astype('bool')
                flux_all=pd.concat([flux_all if not flux_all.empty else None, current_df], ignore_index=True)
    flux_all = flux_all[flux_all["model"]!="model_one_solution"]
    flux_all = flux_all[flux_all["model"]!="model_one_solution"]
    if optim=="submin":
        return flux_all[flux_all["search_mode"]=="Subset Minimal"]
    elif optim=="min":
        return flux_all[flux_all["search_mode"]=="Minimize"]
    else:
        return flux_all

In [12]:
def convert_timers_table(table:pd.DataFrame, optim, list_all_species, dict_species):
    column_conversion = {"REASONING":"Reasoning",
                         "REASONING FILTER":"Filter",
                         "REASONING GUESS-CHECK":'Guess-Check',
                         "REASONING GUESS-CHECK-DIVERSITY":'Guess-Check-div'}

    
    set_complete = set(list_all_species)
    set_reasoning = set(dict_species["reasoning"])
    set_filter = set(dict_species["filter"])
    set_gc = set(dict_species["gc"])
    set_gcd = set(dict_species["gcd"])
    
    diff_reasoning = set_complete.difference(set_reasoning) 
    diff_filter = set_complete.difference(set_filter) 
    diff_gc = set_complete.difference(set_gc) 
    diff_gcd = set_complete.difference(set_gcd) 
    new_table=pd.DataFrame(columns=['network', 'Reasoning','Filter',
                                'Guess-Check', 'Guess-Check-div'])
    new_table = new_table.set_index('network')

    for row in table.iterrows():
        network = row[0]
        if not new_table.empty:
            if network in new_table.index:
                new_table.loc[network, column_conversion[row[1]['mode']]]=row[1][optim]
            else:
                new_table = add_row(new_table, row, optim, column_conversion)
        else:
            new_table = add_row(new_table, row, optim, column_conversion)

    for net in diff_reasoning:
        new_table.loc[net, ["Reasoning"]]=-1000
    for net in diff_filter:
        new_table.loc[net, ["Filter"]]=-1000
    for net in diff_gc:
        new_table.loc[net, ["Guess-Check"]]=-1000
    for net in diff_gcd:
        new_table.loc[net, ["Guess-Check-div"]]=-1000
    return new_table.reset_index()


def add_row(new_table,row, optim, column_conversion):
    network = row[1]['network']
    current = pd.DataFrame(data=[[network, row[1][optim]]], columns=['network', column_conversion[row[1]['mode']]])
    current = current.set_index('network')
    new_table = new_table.combine_first(current)
    return new_table

In [13]:
def get_separate_data(table):
    table["solver_type"] = table["solver_type"].str.replace('REASONING  GUESS-CHECK-DIVERSITY', 'REASONING GUESS-CHECK DIVERSITY')
    table["solver_type"] = table["solver_type"].str.replace('REASONING  GUESS-CHECK', 'REASONING GUESS-CHECK')
    table["solver_type"] = table["solver_type"].str.replace('REASONING  FILTER', 'REASONING FILTER')
    
    # CLASSIC
    table_reasoning = table[table["solver_type"]=="REASONING"]
    
    # FILTER
    table_filter = table[table["solver_type"]=="REASONING FILTER"]

    # GUESS_CHECK
    table_gc = table[table["solver_type"]=="REASONING GUESS-CHECK"]

    # GUESS_CHECK_DIV
    table_gcd = table[table["solver_type"]=="REASONING GUESS-CHECK DIVERSITY"]

    return table_reasoning, table_filter, table_gc, table_gcd

In [14]:
def create_table_plot(table,column_name):
    new_table = table.groupby(['species'])[column_name].agg('count').reset_index()
    new_table=new_table.rename(columns={column_name: "Total_flux"})
    new_true = table[table[column_name]==True].groupby(['species'])[column_name].agg('count').reset_index()
    new_true=new_true.rename(columns={column_name: "True_flux"})
    new_false = table[table[column_name]==False].groupby(['species'])[column_name].agg('count').reset_index()
    new_false=new_false.rename(columns={column_name: "False_flux"})
    new_table=pd.merge(new_table,new_true, how='left', on=['species'])
    new_table=pd.merge(new_table,new_false, how='left', on=['species'])
    new_table=new_table.fillna(0)
    new_table=new_table.fillna(0)
    new_table['True_flux']=new_table['True_flux'].astype(int)
    new_table['False_flux']=new_table['False_flux'].astype(int)
    return new_table

In [15]:
def get_list_species(table):
    labels = table['species'].tolist()
    set_labels = set(labels)
    return set_labels

In [16]:
def get_all_species():
    species_files = os.listdir(sbml_dir)
    species = [sub.replace('.xml', '') for sub in species_files]
    return species

# Get data

In [17]:
timers_FN=get_timers(one_sol_supp_data, "full")
timers_T=get_timers(one_sol_supp_data, "target")

In [18]:
flux_FN = get_fluxes(one_sol_results_dir, "full", "submin")
flux_T = get_fluxes(one_sol_results_dir, "target", "submin")

/tmp/ipykernel_11027/2993539488.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  flux_all=pd.concat([flux_all if not flux_all.empty else None, current_df], ignore_index=True)
/tmp/ipykernel_11027/2993539488.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  flux_all=pd.concat([flux_all if not flux_all.empty else None, current_df], ignore_index=True)
/tmp/ipykernel_11027/2993539488.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a

In [19]:
FN_reasoning, FN_filter, FN_gc, FN_gcd = get_separate_data(flux_FN)

In [20]:
T_reasoning, T_filter, T_gc, T_gcd = get_separate_data(flux_T)

In [21]:
FN_reasoning_tab=create_table_plot(FN_reasoning,'has_flux')
FN_filter_tab=create_table_plot(FN_filter,'has_flux')
FN_gc_tab=create_table_plot(FN_gc,'has_flux')
FN_gcd_tab=create_table_plot(FN_gcd,'has_flux')

T_reasoning_tab=create_table_plot(T_reasoning,'has_flux')
T_filter_tab=create_table_plot(T_filter,'has_flux')
T_gc_tab=create_table_plot(T_gc,'has_flux')
T_gcd_tab=create_table_plot(T_gcd,'has_flux')

In [24]:
dict_species_FN = {"reasoning": get_list_species(FN_reasoning_tab),
                    "filter":get_list_species(FN_filter_tab),
                    "gc":get_list_species(FN_gc_tab),
                    "gcd":get_list_species(FN_gcd_tab)}

dict_species_T = {"reasoning":get_list_species(T_reasoning_tab),
                    "filter":get_list_species(T_filter_tab),
                    "gc":get_list_species(T_gc_tab),
                    "gcd":get_list_species(T_gcd_tab)}

In [25]:
list_all_species = get_all_species()

In [27]:
timers_FN_final = convert_timers_table(timers_FN, "submin",  list_all_species, dict_species_FN)

In [28]:
timers_T_final = convert_timers_table(timers_T, "submin",  list_all_species, dict_species_T)

The plot uses R to get the data tables from `timer_tables_dir` variable, so we are saving data (set at the begining of the notebook in paragraphe  [Variable to change](#variable-to-change-if-wanted)) 

In [29]:
timers_FN_final.to_csv(f"{timer_tables_dir}/Full_Network.tsv", sep='\t')
timers_T_final.to_csv(f"{timer_tables_dir}/Target.tsv", sep='\t')

# PLOT

This part uses R to get the data tables from `timer_tables_dir` variable and write it in `timers_plot_dir` directory (set at the begining of the notebook in paragraphe  [Variable to change](#variable-to-change-if-wanted) ) 

In [30]:
%%R -i timer_tables_dir -i timers_plot_dir
library(reshape2)
library(data.table)
library(ggplot2)


# create output directory if it does not exist
if (!dir.exists(timers_plot_dir)) {
    dir.create(timers_plot_dir)
}

# read all files in the input directory
files <- list.files(timer_tables_dir, full.names = TRUE)

# loop over all files
for (file in files) {
    # read the data
    data <- read.table(file, header = TRUE) # , row.names = 1
    # get the name of the file, no extension
    file_id <- sub(pattern = "(.*)\\..*$", replacement = "\\1", basename(file))
    
    # replace -200 with 2700
    data[data == -200] <- 2700

    # replace -100 with NA
    data[data == -1000] <- NA

    # get long data from wide format, id = row names
    data_long <- reshape2::melt(data, variable.name = "Group", value.name = "Time")


    # plot the data
    p = ggplot(data_long, aes(x = Time, color = Group)) +
        stat_bin(data = subset(data_long, Group == "Filter"), aes(y = cumsum(..count..)), geom = "step") +
        stat_bin(data = subset(data_long, Group == "Guess.Check"), aes(y = cumsum(..count..)), geom = "step") +
        stat_bin(data = subset(data_long, Group == "Guess.Check.div"), aes(y = cumsum(..count..)), geom = "step") +
        stat_bin(data = subset(data_long, Group == "Reasoning"), aes(y = cumsum(..count..)), geom = "step") +
        # style the plot
        theme_bw() +
        labs(title = file_id, x = "Time", y = "Cumulative count of GSMNs with solutions", colour = "Solving mode") +
        # scale_color_manual(values = c("Filter" = "red", "Guess.Check" = "blue", "Guess.Check.div" = "green", "Reasoning" = "black")) +
        # increase font size
        theme(text = element_text(size = 15))

    # save the plot
    ggsave(paste0(timers_plot_dir, "/", file_id, ".pdf"), plot = p, width = 10, height = 7)
}


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


data.table 1.16.0 using 4 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attachement du package : ‘data.table’

Les objets suivants sont masqués depuis ‘package:reshape2’:

    dcast, melt

Using network as id variables
Using network as id variables
De plus : Messages d'avis :
1: Dans (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  les bibliothèques ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ ne contiennent aucun package
2: The dot-dot notation (`..count..`) was deprecated in ggplot2 3.4.0.
ℹ Please use `after_stat(count)` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
3: Removed 3 rows containing non-finite outside the scale range (`stat_bin()`). 
4: Removed 4 rows containing non-finite outside the scale range (`stat_bin()`). 
5: Removed 4 rows containing non-finite outside the scale range (`stat_bin()`). 
6: Removed 1 row containing